In [2]:
import duckdb

In [ ]:
"id": pl.Utf8,
"type": pl.Utf8,
"actor": pl.Struct(
    {
        "id": pl.Int32,
        "login": pl.Utf8,
        "display_login": pl.Utf8,
        "gravatar_id": pl.Utf8,
        "url": pl.Utf8,
        "avatar_url": pl.Utf8,
    }
),
"repo": pl.Struct(
    {
        "id": pl.Int32,
        "name": pl.Utf8,
        "url": pl.Utf8,
    }
),
"public": pl.Boolean,
"created_at": pl.Utf8,
"org": pl.Struct(
    {
        "id": pl.Int32,
        "login": pl.Utf8,
        "gravatar_id": pl.Utf8,
        "url": pl.Utf8,
        "avatar_url": pl.Utf8,
    }
),
"other": pl.Utf8,

In [8]:
columns = {
    'id': 'VARCHAR',
    'type': 'VARCHAR',
    'actor': 'STRUCT(id INTEGER, login VARCHAR, display_login VARCHAR, gravatar_id VARCHAR, url VARCHAR, avatar_url VARCHAR)',
    'repo': 'STRUCT(id INTEGER, name VARCHAR, url VARCHAR)',
    'created_at': 'VARCHAR',
    'org': 'STRUCT(id INTEGER, login VARCHAR, gravatar_id VARCHAR, url VARCHAR, avatar_url VARCHAR)',
}

In [14]:
df = duckdb.sql(
    f"""
    select
        id,
        type,
        actor,
        repo,
        strptime(created_at, '%Y-%m-%dT%H:%M:%SZ') as created_at,
    from read_ndjson('../data/raw/2023/03/2023-03-01-0.json.gz', columns={columns})
    """
)

In [15]:
df.show()

┌─────────────┬──────────────────────┬──────────────────────┬────────────────────────────────────┬─────────────────────┐
│     id      │         type         │        actor         │                repo                │     created_at      │
│   varchar   │       varchar        │ struct(id integer,…  │ struct(id integer, "name" varcha…  │      timestamp      │
├─────────────┼──────────────────────┼──────────────────────┼────────────────────────────────────┼─────────────────────┤
│ 27400475723 │ DeleteEvent          │ {'id': 49699333, '…  │ {'id': 146641150, 'name': tekton…  │ 2023-03-01 00:00:00 │
│ 27400475733 │ IssuesEvent          │ {'id': 1882561, 'l…  │ {'id': 172130649, 'name': cph/Se…  │ 2023-03-01 00:00:00 │
│ 27400475735 │ CreateEvent          │ {'id': 62541748, '…  │ {'id': 607237316, 'name': darthw…  │ 2023-03-01 00:00:00 │
│ 27400475736 │ PushEvent            │ {'id': 8517910, 'l…  │ {'id': 410004573, 'name': Lombiq…  │ 2023-03-01 00:00:00 │
│ 27400475739 │ PushEvent       

In [16]:
df_pl = df.pl()
df_pl.head()

id,type,actor,repo,created_at
str,str,struct[6],struct[3],datetime[μs]
"""27400475723""","""DeleteEvent""","{49699333,""dependabot[bot]"",""dependabot"","""",""https://api.github.com/users/dependabot[bot]"",""https://avatars.githubusercontent.com/u/49699333?""}","{146641150,""tektoncd/pipeline"",""https://api.github.com/repos/tektoncd/pipeline""}",2023-03-01 00:00:00
"""27400475733""","""IssuesEvent""","{1882561,""houstonbot"",""houstonbot"","""",""https://api.github.com/users/houstonbot"",""https://avatars.githubusercontent.com/u/1882561?""}","{172130649,""cph/SearchWithModifiers"",""https://api.github.com/repos/cph/SearchWithModifiers""}",2023-03-01 00:00:00
"""27400475735""","""CreateEvent""","{62541748,""Dominik-Seiler"",""Dominik-Seiler"","""",""https://api.github.com/users/Dominik-Seiler"",""https://avatars.githubusercontent.com/u/62541748?""}","{607237316,""darthweiter/exoplanet"",""https://api.github.com/repos/darthweiter/exoplanet""}",2023-03-01 00:00:00
"""27400475736""","""PushEvent""","{8517910,""LombiqBot"",""LombiqBot"","""",""https://api.github.com/users/LombiqBot"",""https://avatars.githubusercontent.com/u/8517910?""}","{410004573,""Lombiq/TheResumeTheme"",""https://api.github.com/repos/Lombiq/TheResumeTheme""}",2023-03-01 00:00:00
"""27400475739""","""PushEvent""","{122495445,""yanocuangana"",""yanocuangana"","""",""https://api.github.com/users/yanocuangana"",""https://avatars.githubusercontent.com/u/122495445?""}","{587937207,""yanocuangana/wdd130"",""https://api.github.com/repos/yanocuangana/wdd130""}",2023-03-01 00:00:00


In [17]:
duckdb.sql(
    """
    copy df
    to '../2023-03-01-0.parquet' (format parquet)
    """
)